<a href="https://colab.research.google.com/github/0xLighty/Natural-Language-Processing/blob/main/hw4_task3_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 3 
Fine tune the BERT for classification model on the text classification dataset you used in the previous question and check the performance

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import evaluate
from datasets import load_dataset
from transformers import BertTokenizerFast, AdamW, BertForSequenceClassification, get_scheduler
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

In [3]:
# checking the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [4]:
train = load_dataset('tweet_eval', 'emotion', split='train[:2000]')
val = load_dataset('tweet_eval', 'emotion', split='validation[:200]')

In [5]:
# fetching the pretrained tokenizer for BERT
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased',do_lower_case=True)

In [6]:
# creating encodings for training and validation data
train_encodings = tokenizer(train['text'], truncation=True, padding=True,max_length=512)
dev_encodings = tokenizer(val['text'], truncation=True, padding=True,max_length=512)

In [7]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
    def __len__(self):
        return len(self.labels)

In [8]:
train_dataset = TweetDataset(train_encodings, train['label'])
dev_dataset = TweetDataset(dev_encodings, val['label'])

In [9]:
# printing the number of training data
no_of_encodings = len(train_encodings['input_ids'])
print(f'We have {no_of_encodings} examples')

We have 2000 examples


In [17]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(train['label'])))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=8)

In [19]:
# using adam optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)
# no of epochs
num_epochs = 5
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [20]:
# Fine tuning BERT batch wise
progress = tqdm(range(num_training_steps))

# fine tuning the model
model.train()
for epoch in range(num_epochs):
    for batch in train_loader:

        # batch size
        batch = {key: val.to(device) for key, val in batch.items()}

        # model needs to use same device each time
        outputs = model.to(device)(**batch)
        loss = outputs.loss
        loss.backward()

        # optimizer
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        # show progress

        progress.update(1)

100%|██████████| 1250/1250 [02:20<00:00,  8.97it/s]

In [21]:
metric = evaluate.load("accuracy")
model.eval()
for batch in dev_loader:
    batch = {key: val.to(device) for key, val in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits

    # predicting the class using argmax
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.81}